In [1]:
struct F{p, T<:Integer} <: Integer
    a::T
    F{p, T}(a::Integer) where {p, T<:Integer} = new{p, T}(mod(T(a), p))
end
F{p}(a::Integer) where p = F{p, typeof(a)}(mod(a, p))
F{p, S}(x::F{p, T}) where {S<:Integer, p, T<:Integer} = F{p, S}(x.a)

Base.promote_rule(::Type{F{p, T}}, ::Type{S}) where {p, T<:Integer, S<:Integer} =
    F{p, promote_type(T, S)}

Base.zero(::Type{F{p, T}}) where {p, T<:Integer} = F{p}(mod(zero(T), p))
Base.one(::Type{F{p, T}}) where {p, T<:Integer} = F{p}(mod(one(T), p))
for op in (:-, :+)
    @eval Base.$op(x::F{p}) where p = F{p}(mod($op(x.a), p))
end
for op in (:-, :+, :*)
    @eval Base.$op(x::F{p}, y::F{p}) where p = F{p}(mod($op(x.a, y.a), p))
end
Base.inv(x::F{p}) where p = F{p}(invmod(x.a, p))
Base.:/(x::F{p}, y::F{p}) where p = x * inv(y)
Base.:\(x::F{p}, y::F{p}) where p = inv(x) * y
Base.:(==)(x::F{p}, y::F{p}) where p = x.a == y.a
Base.:<(x::F{p}, y::F{p}) where p = x.a < y.a

Base.show(io::IO, x::F{p}) where p = print(io, "F", p, '(', x.a, ')')

F7 = F{7, BigInt}
x, y = F7(10), F7(-2)
@show(x, y, zero(x), one(x), +x, -x, x + y, x - y, x * y, x / y, x \ y, x^3, x^-5, x == F7(3), x == 3)
println()
A = F7[1 2; 3 4]
@show(A, 4A, A/4)
println()
using LinearAlgebra
L, U = lu(A, NoPivot())
@show(L, U, L * U, det(A), inv(A), inv(A) * A, A * inv(A));

x = F7(3)
y = F7(5)
zero(x) = F7(0)
one(x) = F7(1)
+x = F7(3)
-x = F7(4)
x + y = F7(1)
x - y = F7(5)
x * y = F7(1)
x / y = F7(2)
x \ y = F7(4)
x ^ 3 = F7(6)
x ^ -5 = F7(3)
x == F7(3) = true
x == 3 = true

A = F{7, BigInt}[F7(1) F7(2); F7(3) F7(4)]
4A = F{7, BigInt}[F7(4) F7(1); F7(5) F7(2)]
A / 4 = F{7, BigInt}[F7(2) F7(4); F7(6) F7(1)]

L = F{7, BigInt}[F7(1) F7(0); F7(3) F7(1)]
U = F{7, BigInt}[F7(1) F7(2); F7(0) F7(5)]
L * U = F{7, BigInt}[F7(1) F7(2); F7(3) F7(4)]
det(A) = F7(5)
inv(A) = F{7, BigInt}[F7(5) F7(1); F7(5) F7(3)]
inv(A) * A = F{7, BigInt}[F7(1) F7(0); F7(0) F7(1)]
A * inv(A) = F{7, BigInt}[F7(1) F7(0); F7(0) F7(1)]


In [2]:
"""See https://docs.julialang.org/en/v1/manual/interfaces/"""
Base.iterate(Fp::Type{F{p, T}}) where {p, T<:Integer} = (zero(Fp), zero(T))
function Base.iterate(Fp::Type{F{p, T}}, state) where {p, T<:Integer}
    nextstate = state + 1
    nextstate < p ? (Fp(nextstate), nextstate) : nothing
end
Base.IteratorSize(Fp::Type{F{p, T}}) where {p, T<:Integer} = Base.HasLength()
Base.length(Fp::Type{F{p, T}}) where {p, T<:Integer} = p
Base.eltype(Fp::Type{F{p, T}}) where {p, T<:Integer} = Fp

squares(Fp) = Fp[x^2 for x in Fp]
squareroots(k, Fp) = Fp[x for x in Fp if x^2 == k]
@show(collect(F7), squares(F7), squareroots.(0:6, Ref(F7)));

collect(F7) = F{7, BigInt}[F7(0), F7(1), F7(2), F7(3), F7(4), F7(5), F7(6)]
squares(F7) = F{7, BigInt}[F7(0), F7(1), F7(4), F7(2), F7(2), F7(4), F7(1)]
squareroots.(0:6, Ref(F7)) = Vector{F{7, BigInt}}[[F7(0)], [F7(1), F7(6)], [F7(3), F7(4)], [], [F7(2), F7(5)], [], []]


In [3]:
using GaloisFields, LinearAlgebra
GF7 = @GaloisField 7
B = GF7[1 2; 3 4]
det(B)

LoadError: MethodError: no method matching abs(::𝔽₇)
[0mClosest candidates are:
[0m  abs([91m::Unsigned[39m) at int.jl:173
[0m  abs([91m::Signed[39m) at int.jl:174
[0m  abs([91m::Complex[39m) at complex.jl:277
[0m  ...

In [4]:
inv(B)

LoadError: MethodError: no method matching abs(::𝔽₇)
[0mClosest candidates are:
[0m  abs([91m::Unsigned[39m) at int.jl:173
[0m  abs([91m::Signed[39m) at int.jl:174
[0m  abs([91m::Complex[39m) at complex.jl:277
[0m  ...

In [5]:
lu(B)

LoadError: MethodError: no method matching abs(::𝔽₇)
[0mClosest candidates are:
[0m  abs([91m::Unsigned[39m) at int.jl:173
[0m  abs([91m::Signed[39m) at int.jl:174
[0m  abs([91m::Complex[39m) at complex.jl:277
[0m  ...

In [6]:
using AbstractAlgebra

@show GF7 = GF(7)
x, y = GF7(10), GF7(-2)
@show(x, y, zero(x), one(x), -x, x + y, x - y, x * y, x^3, x^-5, x == GF7(3), x == 3)
println()

@show C = GF7[1 2; 3 4]
@show P, x = PolynomialRing(GF7, "x")
@show(det(C), inv(C), lu(C), charpoly(P, C))
println()

squares(Fp) = [x^2 for x in Fp]
squareroots(k, Fp) = [x for x in Fp if x^2 == k]
@show(collect(GF7), squares(GF7), squareroots.(0:6, Ref(GF7)));

GF7 = GF(7) = Finite field F_7
x = 3
y = 5
zero(x) = 0
one(x) = 1
-x = 4
x + y = 1
x - y = 5
x * y = 1
x ^ 3 = 6
x ^ -5 = 3
x == GF7(3) = true
x == 3 = true

C = GF7[1 2; 3 4] = [1 2; 3 4]
(P, x) = PolynomialRing(GF7, "x") = (Univariate Polynomial Ring in x over Finite field F_7, x)
det(C) = 5
inv(C) = [5 1; 5 3]
lu(C) = (2, (), [1 0; 3 1], [1 2; 0 5])
charpoly(P, C) = x^2 + 2*x + 5

collect(GF7) = AbstractAlgebra.GFElem{Int64}[0, 1, 2, 3, 4, 5, 6]
squares(GF7) = AbstractAlgebra.GFElem{Int64}[0, 1, 4, 2, 2, 4, 1]
squareroots.(0:6, Ref(GF7)) = Vector{AbstractAlgebra.GFElem{Int64}}[[0], [1, 6], [3, 4], [], [2, 5], [], []]
